# Custom WCS System for Goodman Spectroscopic Pipeline

In [30]:
"""
cfzsto_0084_CuHeAr_G1200M2_slit103.fits
Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 2
Parameters:
          c0            c1               c2        
    ------------- -------------- ------------------
    4237.08542489 0.325202190503 -4.84559634299e-07
"""

from astropy.modeling import models, fitting, Model
import astropy.units as u
from ccdproc import CCDData

_model = models.Chebyshev1D(degree=2, c0=4237.08542489, c1=0.325202190503, c2=-4.84559634299e-07)

print(_model.__class__.name)

print(_model.degree, _model.c0.value,_model.c0.value,_model.c0.value)

keywords = {'GSP_FUNC':_model.__class__.name,
            'GSP_ORDR': _model.degree,
            'GSP_C000': _model.c0.value,
            'GSP_C001': _model.c0.value,
            'GSP_C002': _model.c0.value,
            'GSP_NPIX': 4800}

# print(_model.__getattr__('c0'))

def wcs_gsp_to_fits(ccd, model):
    assert isinstance(ccd, CCDData)
    assert isinstance(model, Model)
    
    ccd.header.set('GSP_FUNC', value=model.__class__.name, comment="Mathematical model")
    ccd.header.set('GSP_ORDR', value=model.degree, comment="Mathematical model order")
    ccd.header.set('GSP_NPIX', value=ccd.size, comment="Mathematical model order")
    for i in range(model.degree + 1):
        ccd.header.set('GSP_C{:03d}'.format(i),
                       value=model.__getattr__('c{:d}'.format(i)).value,
                       comment="Value of parameter c{:d}".format(i))
    return ccd

def gsp_wcs_read(ccd):
    assert isinstance(ccd, CCDData)
    
    if ccd.header['GSP_FUNC'] == 'Chebyshev1D':
        model = models.Chebyshev1D(degree=ccd.header['GSP_ORDR'])
        for i in range(ccd.header['GSP_ORDR'] + 1):
            model.__getattr__('c{:d}'.format(i)).value = ccd.header['GSP_C{:03d}'.format(i)]
        return model

    
    
    
ccd = CCDData.read('./data/cfzsto_0084_CuHeAr_G1200M2_slit103.fits', unit=u.adu)
                       
nccd = wcs_gsp_to_fits(ccd=ccd, model=_model)

# for key in nccd.header.keys():
#     print("{:8s}= {:s}".format(key, str(nccd.header[key])))
nmodel = gsp_wcs_read(ccd=nccd)
print(nmodel)

Chebyshev1D
2 4237.08542489 4237.08542489 4237.08542489
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 2
Parameters:
          c0            c1               c2        
    ------------- -------------- ------------------
    4237.08542489 0.325202190503 -4.84559634299e-07
